In [2]:
# Imports
import ROOT
import os
import numpy as np
from array import array

# Path to the data file (update if needed)
fname = "/Users/snip/Documents/MEOP/0430_0.022V_2T_f4+ -1.2mbar_0.6W.csv"
print('Data file:', fname)

Data file: /Users/snip/Documents/MEOP/0430_0.022V_2T_f4+ -1.2mbar_0.6W.csv


In [5]:
# Load the data robustly and create ROOT objects (TH1D or TGraph) then fit.
# This cell attempts to detect 1-column (histogram) or 2-column (x,y) data.
data = np.loadtxt(fname, delimiter=',', comments='#')
print('Loaded data shape:', getattr(data, 'shape', None))

# Single column -> histogram
if data.ndim == 1 or (data.ndim == 2 and data.shape[1] == 1):
    # Ensure a 1-D numpy array
    arr = data.flatten()
    n_bins = min(200, max(10, int(len(arr)/10)))
    h = ROOT.TH1D('h1', 'Data histogram;Value;Counts', n_bins, float(arr.min()), float(arr.max()))
    for v in arr:
        h.Fill(float(v))
    c = ROOT.TCanvas('c1','c1',800,600)
    h.Draw()
    # Try a gaussian fit first, fallback to a polynomial if it fails
    try:
        res = h.Fit('gaus','S')
        print('Gaussian fit status:', res.Status())
        params = [res.Parameter(i) for i in range(res.NPar())]
        print('Fit parameters:', params)
    except Exception as e:
        print('Gaussian fit failed:', e)
        try:
            res2 = h.Fit('pol1','S')
            print('Linear fit status:', res2.Status())
            print('Linear params:', [res2.Parameter(i) for i in range(res2.NPar())])
        except Exception as e2:
            print('Polynomial fit failed:', e2)
    c.Draw()
    out = 'fit_hist.png'
    c.SaveAs(out)
    print('Saved plot to', out)

else:
    # Expecting at least two columns; use first two as x,y
    if data.ndim == 2 and data.shape[1] >= 2:
        x = data[:,0]
        y = data[:,1]
    else:
        raise ValueError('Unexpected data shape: ' + str(getattr(data, 'shape', None)))
    n = len(x)
    ax = array('d', x.astype(float).tolist())
    ay = array('d', y.astype(float).tolist())
    g = ROOT.TGraph(n, ax, ay)
    g.SetTitle('Data;X;Y')
    c = ROOT.TCanvas('c1','c1',800,600)
    g.Draw('APL')
    # Try gaussian fit (suitable for peaked data) then fallback to linear
    try:
        f1 = ROOT.TF1('f1','gaus')
        res = g.Fit(f1,'S')
        print('Gaussian fit status:', res.Status())
        print('Parameters:', [res.Parameter(i) for i in range(res.NPar())])
    except Exception as e:
        print('Gaussian fit failed:', e)
        try:
            f2 = ROOT.TF1('f2','pol1')
            res2 = g.Fit(f2,'S')
            print('Linear fit status:', res2.Status())
            print('Linear params:', [res2.Parameter(i) for i in range(res2.NPar())])
        except Exception as e2:
            print('Linear fit failed:', e2)
    c.Draw()
    out = 'fit_graph.png'
    c.SaveAs(out)
    print('Saved plot to', out)

Loaded data shape: (515, 2)
Gaussian fit status: 3
Parameters: [89.1762323352863, 1746037882.5330465, 511.5082073055413]
Saved plot to fit_graph.png
****************************************
         Invalid FitResult  (status = 3 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      53517.4
NDf                       =          512
Edm                       =      1251.64
NCalls                    =          201
Constant                  =      89.1762   +/-   1.11912     
Mean                      =  1.74604e+09   +/-   8.84474     
Sigma                     =      511.508   +/-   7.95251      	 (limited)
****************************************
         Invalid FitResult  (status = 3 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      53517.4
NDf                       =          512
Edm                       =      1251.64
NCalls                    =          201
Constant         

Warning in <Fit>: Abnormal termination of minimization.
Info in <TCanvas::Print>: png file fit_graph.png has been created


In [8]:
# Apply smoothing filter to remove outliers
from scipy.signal import savgol_filter
from scipy.ndimage import median_filter

# Extract timestamp and values
x_raw = data[:, 0]
y_raw = data[:, 1]

# Apply Savitzky-Golay filter (polynomial smoothing)
# window_length must be odd and less than data length
window_length = min(51, len(y_raw) if len(y_raw) % 2 == 1 else len(y_raw) - 1)
if window_length < 5:
    window_length = 5
polyorder = 3

y_smooth = savgol_filter(y_raw, window_length, polyorder)

# Calculate residuals (deviation from smooth curve)
residuals = np.abs(y_raw - y_smooth)
threshold = np.mean(residuals) + 3 * np.std(residuals)  # Remove values > 3 sigma from smoothed curve

# Create mask for valid points (not outliers)
valid_mask = residuals < threshold

print(f"Original points: {len(y_raw)}")
print(f"Outliers removed: {len(y_raw) - np.sum(valid_mask)}")
print(f"Residual threshold: {threshold:.4f}")

# Filter data to remove outliers
data_filtered = data[valid_mask]
print(f"Filtered data shape: {data_filtered.shape}")


Original points: 515
Outliers removed: 10
Residual threshold: 19.2115
Filtered data shape: (505, 2)
